In [2]:
section_text = """
Informationssicherheit – SoSe 2025
Organisatorisches & Motivation
"""


In [3]:
prompt = f"""
You are a research assistant. Summarize the text below in max 5 bullet points.
Focus on: problem addressed, methods, results, limitations.
Be concise and avoid generic filler.

Text:
{section_text}
"""


In [4]:
question = "What datasets does the paper use?"

prompt = f"""
You are a research assistant. Answer the question below using ONLY the provided text.
If the answer is not explicitly in the text, respond exactly: "Not found."

Question: {question}

Context:
{section_text}
"""


In [5]:
def offline_generate(text):
    # Zeigt einfach die wichtigsten Sätze
    sents = text.split("\n")
    return "\n".join(sents[:5])

print(offline_generate(section_text))



Informationssicherheit – SoSe 2025
Organisatorisches & Motivation



In [7]:
%pip install sentence-transformers faiss-cpu PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import fitz  # PyMuPDF

doc = fitz.open("data/papers/infoSicherheit_merged.pdf")
text = ""
for page in doc:
    text += page.get_text("text")
print(text[:500])  # Vorschau


Informationssicherheit – SoSe 2025
Organisatorisches & Motivation
Prof. Dr. Holger Schmidt
holger.schmidt004[at]fh-dortmund.de
Fachhochschule Dortmund
Fachbereich Informatik
Professur für IT-Sicherheit, Informatik
1. April 2025, 10:20:47
1 / 41
IT-Sicherheit, Informatik
2 / 41
Professur für IT-Sicherheit, Informatik I
▶Prof. Dr. Holger Schmidt
▶Informationssicherheit (46813)
(alter Titel: Datenschutz und Datensicherheit)
▶Informatik (Bachelor, 4. Sem., Pflicht)
▶Informatik Dual (Bachelor, 2. Sem


In [9]:
def chunk_text(text, size=500, overlap=100):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + size
        chunks.append(" ".join(words[start:end]))
        start = end - overlap
    return chunks

chunks = chunk_text(text)
print("Anzahl Chunks:", len(chunks))


Anzahl Chunks: 51


In [10]:
from sentence_transformers import SentenceTransformer
import faiss, numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks, normalize_embeddings=True)

index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(np.array(embeddings, dtype="float32"))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\joel2\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joel2\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
query = "worum geht es hier ?"
q_emb = model.encode([query], normalize_embeddings=True)
D, I = index.search(np.array(q_emb, dtype="float32"), k=3)

for idx in I[0]:
    print("Treffer:", chunks[idx][:200], "...")


Treffer: k ∈G (Assoziativgesetz), G2 g • e = e • g = g für alle g ∈G (Neutrales Element) und G3 Zu jedem g ∈G gibt es ein h ∈G mit g • h = h • g = e (Inverses Element). Gilt außerdem g • h = h • g für alle g,  ...
Treffer: ...“10. ▶Identifikation passiert durch „...direkt oder indirekt, insbesondere mittels Zuordnung zu einer Kennung wie einem Namen, zu einer Kennnummer, zu Standortdaten, zu einer Online-Kennung oder zu ...
Treffer: die dann keinerlei äußere Einwirkung mehr benötigen. Angst wird häufig durch das Vortäuschen einer besonders kritischen Situation (Notfallsituation) hervorgerufen und verleitet z. B. zu schadhaften Ha ...
